In [1]:
import re
import numpy as np
import pandas as pd

## Read readings file

In [2]:
with open('senario1_readings30.txt') as f1:
    lines_sen_01 = f1.readlines()
with open('senario2_readings30.txt') as f2:
    lines_sen_02 = f2.readlines()
with open('senario1_readings2.txt') as f3:
    lines_sen_03 = f3.readlines()
with open('senario2_readings2.txt') as f4:
    lines_sen_04 = f4.readlines()    

## Extract readings in dataframe

In [3]:
# list to dictionary
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

def prepare_data(lines):
    sensors_reading = lines[54:]
    controller = []

    for i in range(len(sensors_reading)):
        if re.search(r'\ Controller\b', sensors_reading[i]):
            controller.append(sensors_reading[i])
    lst = []
    for i in range(len(controller)):
        lst.append(controller[i][37: -2].split())

    reading_lst = []
    for i in range(len(lst)):
        reading_lst.append(Convert(lst[i]))
    df = pd.DataFrame(reading_lst)    
    
    df.rename(columns={'Temperature:': 'Temperature', 'Humidity:': 'Humidity', 'Pressure:': 'Pressure'}, inplace=True)
    df['Temperature'] = df['Temperature'].str.replace(',',' ')
    df['Humidity'] = df['Humidity'].str.replace(',',' ')

    df["Temperature"] = pd.to_numeric(df["Temperature"], downcast="float")
    df["Humidity"] = pd.to_numeric(df["Humidity"], downcast="float")
    df["Pressure"] = pd.to_numeric(df["Pressure"], downcast="float")
    
    return df

In [4]:
df1 = prepare_data(lines_sen_01)
df2 = prepare_data(lines_sen_02)
df3 = prepare_data(lines_sen_03)
df4 = prepare_data(lines_sen_04)

## Calculate the mean value for each round 

In [6]:
def calc_mean(df):
    temp_5, humidity_5, pressure_5 = [], [] , []

    for i in range(0, len(df), 5):
        temp_5.append(np.mean(df['Temperature'].iloc[i: i + 5]))
        humidity_5.append(np.mean(df['Humidity'].iloc[i: i + 5]))
        pressure_5.append(np.mean(df['Pressure'].iloc[i: i + 5]))
    return temp_5, humidity_5, pressure_5  

In [7]:
# Senario 01 round 30
temp01, humidity01, pressure01 = calc_mean(df1)
# Senario 02 round 30
temp02, humidity02, pressure02 = calc_mean(df2)
# Senario 01 round 2
temp03, humidity03, pressure03 = calc_mean(df3)
# Senario 02 round 2
temp04, humidity04, pressure04 = calc_mean(df4)

## Add noise to data according to normal gussian distribution

### Senario 1 round 30

In [8]:
mu, sigma = 0, 1 # mean and standard deviation
noise_data1 = np.random.normal(mu, sigma, 30)

temp_noise1, humadity_noise1, pressure_noise1 = [], [], []

for i in range(len(temp01)):
    temp_noise1.append(temp01[i] + noise_data1[i])
    humadity_noise1.append(humidity01[i] + noise_data1[i])
    pressure_noise1.append(pressure01[i] + noise_data1[i])

### Senario 2 round 30

In [9]:
mu, sigma = 0, 1 # mean and standard deviation
noise_data2 = np.random.normal(mu, sigma, 114)

temp_noise2, humadity_noise2, pressure_noise2 = [], [], []

for i in range(len(temp02)):
    temp_noise2.append(temp02[i] + noise_data2[i])
    humadity_noise2.append(humidity02[i] + noise_data2[i])
    pressure_noise2.append(pressure02[i] + noise_data2[i])

### Senario 1 round 2

In [10]:
noise_data3 = np.random.normal(mu, sigma, 2)

temp_noise3, humadity_noise3, pressure_noise3 = [], [], []

for i in range(len(temp03)):
    temp_noise3.append(temp03[i] + noise_data3[i])
    humadity_noise3.append(humidity03[i] + noise_data3[i])
    pressure_noise3.append(pressure03[i] + noise_data3[i])

### Senario 2 round 2

In [11]:
noise_data4 = np.random.normal(mu, sigma, 9)

temp_noise4, humadity_noise4, pressure_noise4 = [], [], []

for i in range(len(temp04)):
    temp_noise4.append(temp04[i] + noise_data4[i])
    humadity_noise4.append(humidity04[i] + noise_data4[i])
    pressure_noise4.append(pressure04[i] + noise_data4[i])

### Estimated Error senario 1 round 30

In [12]:
true_readings = [25.0, 40.0, 101000.0]
error_temp1, error_humadity1, error_pressure1 = [], [], []
for i in range(len(temp_noise1)):
    error_temp1.append(np.absolute(temp_noise1[i] - true_readings[0]))
    error_humadity1.append(np.absolute(humadity_noise1[i] - true_readings[1]))
    error_pressure1.append(np.absolute(pressure_noise1[i] - true_readings[2]))

In [13]:
print("Average Temperature Error in Scenario 1: {:.3f}".format(np.mean(error_temp1)))
print("Average Humadity Error in Scenario 1   : {:.3f}".format(np.mean(error_humadity1)))
print("Average Pressure Error in Scenario 1   : {:.3f}".format(np.mean(error_pressure1)))

Average Temperature Error in Scenario 1: 3.298
Average Humadity Error in Scenario 1   : 5.172
Average Pressure Error in Scenario 1   : 12120.336


### Estimated Error senario 2 round 30

In [25]:
error_temp2, error_humadity2, error_pressure2 = [], [], []
for i in range(len(temp_noise2)):
    error_temp2.append(np.absolute(temp_noise2[i] - true_readings[0]))
    error_humadity2.append(np.absolute(humadity_noise2[i] - true_readings[1]))
    error_pressure2.append(np.absolute(pressure_noise2[i] - true_readings[2]))

In [26]:
print("Average Temperature Error in Scenario 2: {:.3f}".format(np.mean(error_temp2)))
print("Average Humadity Error in Scenario 2   : {:.3f}".format(np.mean(error_humadity2)))
print("Average Pressure Error in Scenario 2   : {:.3f}".format(np.mean(error_pressure2)))

Average Temperature Error in Scenario 2: 1.326
Average Humadity Error in Scenario 2   : 1.991
Average Pressure Error in Scenario 2   : 5049.936


### Estimated Error senario 1 round 2

In [27]:
error_temp3, error_humadity3, error_pressure3 = [], [], []
for i in range(len(temp_noise3)):
    error_temp3.append(np.absolute(temp_noise3[i] - true_readings[0]))
    error_humadity3.append(np.absolute(humadity_noise3[i] - true_readings[1]))
    error_pressure3.append(np.absolute(pressure_noise3[i] - true_readings[2]))

### Estimated Error senario 2 round 2

In [28]:
error_temp4, error_humadity4, error_pressure4 = [], [], []
for i in range(len(temp_noise4)):
    error_temp4.append(np.absolute(temp_noise4[i] - true_readings[0]))
    error_humadity4.append(np.absolute(humadity_noise4[i] - true_readings[1]))
    error_pressure4.append(np.absolute(pressure_noise4[i] - true_readings[2]))

#### Method to calculate min, max, mean values

In [29]:
def min_max_avg (err_list):
    avg_val = np.mean(err_list)
    min_val = np.min(err_list)
    max_val = np.max(err_list)
    print("Max Val: {:.2f}".format(max_val))
    print("Min Val: {:.2f}".format(min_val))
    print("Avg Val: {:.2f}".format(avg_val))
    return avg_val , min_val , max_val

## Round 30

#### Temperature Senario 01 round 30

In [30]:
avg_temp1 , min_temp1 , max_temp1 = min_max_avg(error_temp1)

Max Val: 5.75
Min Val: 0.78
Avg Val: 3.30


#### Humadity Senario 01 round 30

In [31]:
avg_humadity1 , min_humadity1 , max_humadity1 = min_max_avg(error_humadity1)

Max Val: 7.42
Min Val: 2.41
Avg Val: 5.17


#### Pressure Senario 01 round 30

In [32]:
avg_pressure1 , min_pressure1 , max_pressure1 = min_max_avg(error_pressure1)

Max Val: 12123.16
Min Val: 12118.10
Avg Val: 12120.34


#### Temperature Senario 02 round 30

In [33]:
avg_temp2 , min_temp2 , max_temp2 =  min_max_avg(error_temp2)

Max Val: 3.45
Min Val: 0.01
Avg Val: 1.33


#### Humadity Senario 02 round 30

In [34]:
avg_humadity2 , min_humadity2 , max_humadity2 = min_max_avg(error_humadity2)

Max Val: 4.59
Min Val: 0.15
Avg Val: 1.99


#### Pressure Senario 02 round 30

In [35]:
avg_pressure2 , min_pressure2 , max_pressure2 = min_max_avg(error_pressure2)

Max Val: 5052.30
Min Val: 5047.25
Avg Val: 5049.94


## Round 2

#### Temperature Senario 01 round 2

In [36]:
avg_temp3 , min_temp3 , max_temp3 = min_max_avg(error_temp3)

Max Val: 3.21
Min Val: 2.02
Avg Val: 2.62


#### Humadity Senario 01 round 2

In [37]:
avg_hum3 , min_hum3 , max_hum3 = min_max_avg(error_humadity3)

Max Val: 3.89
Min Val: 3.66
Avg Val: 3.78


#### Pressure Senario 01 round 2

In [38]:
avg_pressure3 , min_pressure3 , max_pressure3 = min_max_avg(error_pressure3)

Max Val: 12119.66
Min Val: 12119.44
Avg Val: 12119.55


#### Temperature Senario 02 round 2

In [39]:
avg_temp4 , min_temp4 , max_temp4 = min_max_avg(error_temp4)

Max Val: 3.14
Min Val: 0.38
Avg Val: 1.67


#### Humadity Senario 02 round 2

In [40]:
avg_hum4 , min_hum4 , max_hum4 = min_max_avg(error_humadity4)

Max Val: 3.59
Min Val: 0.20
Avg Val: 1.71


#### Pressure Senario 02 round 2

In [41]:
avg_pressure4 , min_pressure4 , max_pressure4 = min_max_avg(error_pressure4)

Max Val: 5051.38
Min Val: 5047.35
Avg Val: 5049.76
